In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

# URL for GameStop revenue data (Macrotrends)
url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"

# Send HTTP request with headers to mimic a browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response = requests.get(url, headers=headers)
html_data = response.text

# Parse HTML
soup = BeautifulSoup(html_data, "html.parser")

# Try different approaches to find revenue data
# Approach 1: Look for tables with specific text in headers or nearby elements
revenue_data = []
dates = []
revenues = []

# Look for all tables
all_tables = soup.find_all('table')
print(f"Found {len(all_tables)} tables on the page")

# Try to find tables with financial data
for i, table in enumerate(all_tables):
    print(f"Examining table {i+1}...")
    
    # Check table headers or content for revenue-related terms
    if any(term in table.text.lower() for term in ['revenue', 'quarterly', 'annual', 'gamestop']):
        print(f"Table {i+1} contains relevant keywords")
        
        # Extract rows
        rows = table.find_all('tr')
        
        for row in rows[1:]:  # Skip header row
            cols = row.find_all('td')
            if len(cols) >= 2:  # Ensure there are at least 2 columns
                date = cols[0].text.strip()
                revenue = cols[1].text.strip()
                
                # Check if this looks like a date and revenue
                if re.match(r'\d{4}-\d{2}-\d{2}|\w+ \d{4}|\d{4}', date) and re.search(r'\d', revenue):
                    dates.append(date)
                    revenues.append(revenue)
                    print(f"Found data: {date} - {revenue}")

# If we couldn't find data in tables, try looking for structured data in scripts
if not dates:
    print("Trying alternative approach: looking for data in scripts...")
    scripts = soup.find_all('script')
    for script in scripts:
        if script.string and 'quarterly' in script.string.lower() and 'revenue' in script.string.lower():
            # Try to extract JSON data
            match = re.search(r'var\s+originalData\s*=\s*(\[.*?\]);', script.string, re.DOTALL)
            if match:
                import json
                try:
                    data = json.loads(match.group(1))
                    for item in data:
                        if 'date' in item and 'revenue' in item:
                            dates.append(item['date'])
                            revenues.append(item['revenue'])
                except json.JSONDecodeError:
                    print("Found potential data but couldn't parse JSON")

# Create dataframe if we have data
if dates and revenues:
    print(f"Successfully extracted {len(dates)} data points")
    gme_revenue = pd.DataFrame({"Date": dates, "Revenue": revenues})
    
    # Clean data (remove commas, $ signs, and empty values)
    gme_revenue["Revenue"] = gme_revenue["Revenue"].str.replace(r'[\$,]', '', regex=True)
    gme_revenue = gme_revenue[gme_revenue["Revenue"] != ""]
    
    # Convert to numeric
    gme_revenue["Revenue"] = pd.to_numeric(gme_revenue["Revenue"], errors='coerce')
    gme_revenue = gme_revenue.dropna()
    
    # Display last 5 rows
    print("\nGameStop Revenue - Last 5 Rows:")
    print(gme_revenue.tail())
else:
    print("\nCould not extract revenue data. The website structure has likely changed significantly.")
    print("Consider using an official financial data API instead of web scraping for more reliable results.")
    print("Alternatives: Yahoo Finance API, Alpha Vantage, or Financial Modeling Prep API")

Found 6 tables on the page
Examining table 1...
Table 1 contains relevant keywords
Found data: 2025 - $3,823
Found data: 2024 - $5,273
Found data: 2023 - $5,927
Found data: 2022 - $6,011
Found data: 2021 - $5,090
Found data: 2020 - $6,466
Found data: 2019 - $8,285
Found data: 2018 - $8,547
Found data: 2017 - $7,965
Found data: 2016 - $9,364
Found data: 2015 - $9,296
Found data: 2014 - $9,040
Found data: 2013 - $8,887
Found data: 2012 - $9,551
Found data: 2011 - $9,474
Found data: 2010 - $9,078
Found data: 2009 - $8,806
Examining table 2...
Table 2 contains relevant keywords
Found data: 2025-04-30 - $732
Found data: 2025-01-31 - $1,283
Found data: 2024-10-31 - $860
Found data: 2024-07-31 - $798
Found data: 2024-04-30 - $882
Found data: 2024-01-31 - $1,794
Found data: 2023-10-31 - $1,078
Found data: 2023-07-31 - $1,164
Found data: 2023-04-30 - $1,237
Found data: 2023-01-31 - $2,226
Found data: 2022-10-31 - $1,186
Found data: 2022-07-31 - $1,136
Found data: 2022-04-30 - $1,378
Found data: